In [1]:
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup
import re

In [2]:
def get_details(link):
    haha = req.get(link)
    html_content = haha.text
    soup = BeautifulSoup(html_content, 'html.parser')
    title_raw = soup.find('h1', id="course_preview_title").text.strip()
    
    # Modified regex pattern to capture the entire title
    match = re.match(r'(\S+\s\d+)\s+(.*?)(?:\s*\(\d+\s+credits\))?$', title_raw)
    if match:
        course_code = match.group(1)  # e.g., "MUS 109"
        course_title = match.group(2).strip()  # Title text
    else:
        course_code = ""
        course_title = ""
    
    h1_element = soup.find('h1', id='course_preview_title')
    hours_list = []
    for sibling in h1_element.find_next_siblings():
        if sibling.name == 'em':
            hours_list.append(sibling.text.strip())
        elif sibling.name == 'br':
            break  # Stop when we hit a <br> tag
    hours = ', '.join(hours_list)
    #prereq
    em_tags = soup.find_all('em')
    prerequisites = []
    other = []
    coreq = []
    other_coreq = []
    for i, em in enumerate(em_tags):
        if "Prerequisite(s):" in em.text:
            # Start collecting <a> tags and subsequent text
            for sibling in em.find_all_next():
                if sibling.name == 'em' and sibling.text.strip() == "Taught:":
                    break
                elif sibling.name == 'a':
                    prerequisites.append(sibling.text.strip())  # Add <a> text     
                elif sibling.name == 'em':
                    other.append(sibling.text.strip())  # Add <a> text 
        if "Co-requisite(s):" in em.text:
             for sibling in em.find_all_next():
                if sibling.name == 'em' and sibling.text.strip() == "Taught:":
                    break
                elif sibling.name == 'a':
                    coreq.append(sibling.text.strip())  # Add <a> text     
                elif sibling.name == 'em':
                    other_coreq.append(sibling.text.strip())  # Add <a> tex

    return course_code, course_title, hours, prerequisites,other,coreq,other_coreq

In [3]:
print(get_details("http://nku.catalog.acalog.com/preview_course_nopop.php?catoid=23&coid=61739"))

('WLL 396', 'World Languages and Literatures Internship  (3 Credits)', 'Hours:, 3 classroom + 0 lab/studio', [], ['Written permission of supervising faculty member; declared major or minor in World Languages and Literatures; B or better in a 300-level course in target language; GPA of at least 2.75.'], [], [])


In [4]:
course_data = []
with open('Data_Collection\filtered_file.txt', 'r') as file:
    urls = file.readlines()
    for url in urls:
        url = url.strip()
        course_code,course_title,hours,prerequisites,other,coreq,other_coreq  = get_details(url)
        course_data.append({
            'Course_Code': course_code,
            'Course_Title': course_title,
            'Hours': hours,
            'Prerequisites': prerequisites,  # Join prerequisites as a string
            'Other_Pre-req':' '.join(other),
            'Coreq': coreq,
            'Other_Co-req': ','.join(other_coreq)
        })

In [5]:
course_df = pd.DataFrame(course_data)
course_df.to_csv('course_details.csv', index=False)